In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read in dataset

In [ ]:
import pandas as pd
import glob
import os
import numpy as np

d2 = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")

<ipython-input-2-e537f9d417c7>:6: DtypeWarning: Columns (27,92,93,94,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")


# Prepares & Cleans dataset

In [ ]:
#This will ignore all DeprecationWarning warnings in your code.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
cleandf = df2[df2['TaggedPitchType'].str.contains("Undefined") == False]
cleanD1 = cleandf[cleandf['Level'] == "D1"]
df = cleanD1[cleanD1['PitchCall'].str.contains("InPlay") == True]
# df3 = BIPdf.dropna(subset=['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'pfxx', 'pfxz', 'VertApprAngle', 'HorzApprAngle'])

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

# makes outcome dataset that only contains singles
##outcome = df.query("PlayResult =='Single'")

# creates list of conditions that determine whether PlayResult is a hit or not
conditions = [
    (df['PlayResult'] == 'Single'),
    (df['PlayResult'] == 'Double'),
    (df['PlayResult'] == 'Triple'),
    (df['PlayResult'] == 'HomeRun'),
    (df['PlayResult'] == 'Out'),
    (df['PlayResult'] == 'Error'),
    (df['PlayResult'] == 'Sacrifice'),
    (df['PlayResult'] == 'FieldersChoice')
]
# creates a list of values to assign for hit (1) or not (0)
values = [1, 2, 3, 4, 0, 0, 0, 0]
# creates new column determining hit or not and assign values to it based on the conditions
df['hitType'] = np.select(conditions, values)

In [ ]:
# Recodes pitch types into 5 different buckets

# sinkers, 4 seam, 2 seam, 1 seam = fastball
# cutters = sliders
# splitter = changeup

# creates list of conditions that determine whether TaggedPitchType is fastball/slider/changeup/curveball
conditions = [
    (df['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker')),  # fastballs
    (df['TaggedPitchType'].str.contains('Slider|Cutter')),  # sliders
    (df['TaggedPitchType'].str.contains('ChangeUp|Splitter')),  # changeups
    (df['TaggedPitchType'].str.contains('Curveball')) # curveballs
]

# creates a list of values to assign for pitch types
values = [0, 1, 2, 3]
# creates new column determining pitch type and assign values to it based on the conditions; assigns 4 if an other pitch
df['pitchType'] = np.select(conditions, values, default = 4)

In [ ]:
# Recodes Pitch Handedness to binary variable

conditions = [
    (df['PitcherThrows'] == 'Right'),  # RHP
    (df['PitcherThrows'] == 'Left')  # LHP
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
df['pitcherHand'] = np.select(conditions, values)

In [ ]:
# Recodes Hitting Handedness to binary variable

conditions = [
    (df['BatterSide'] == 'Right'),  # RHB
    (df['BatterSide'] == 'Left')  # LHB
]

# creates a list of values to assign for hitter handedness where RHB = 1 & LHB = 0
values = [0, 1]
# creates new column determining hit hand
df['hitterHand'] = np.select(conditions, values)

In [ ]:
# Calculates x & y position of the ball hit
df['xSpray'] = np.sin(np.radians(df['Bearing'])) * df['Distance']
df['ySpray'] = np.cos(np.radians(df['Bearing'])) * df['Distance']

In [ ]:
df['hitType'].value_counts()

1    1062310
2     454762
3     223975
4     138948
Name: pitchType, dtype: int64

# Following Code Identifies Important Variables

Finds correlations between variables to reduce multicollinearity

In [ ]:
# Use to determine multicollinearity
df4 = df[['pitchType', 'pitcherHand', 'hitterHand', 'ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'HitSpinRate', 'Distance', 'Bearing', 'HangTime', 'MaxHeight', 'ContactPositionX', 'ContactPositionY', 'ContactPositionZ', 'isHit']].dropna()
df4.corr()

,pitchType,pitcherHand,RelSpeed,SpinRate,InducedVertBreak,HorzBreak,SpinAxis,pfxx,pfxz,VertApprAngle,HorzApprAngle
pitchType,1.000000,0.001030,-0.730441,-0.097954,-0.708417,-0.205131,-0.242976,0.196533,-0.702326,-0.662862,-0.121797
pitcherHand,0.001030,1.000000,-0.106424,-0.102145,0.031345,-0.449627,-0.040127,0.514619,0.026283,-0.002670,0.580194
RelSpeed,-0.730441,-0.106424,1.000000,0.083108,0.708681,0.315139,0.293251,-0.308828,0.716080,0.672893,0.117397
SpinRate,-0.097954,-0.102145,0.083108,1.000000,-0.147132,-0.148164,-0.218405,0.132162,-0.144478,-0.050363,-0.195104
InducedVertBreak,-0.708417,0.031345,0.708681,-0.147132,1.000000,0.240898,0.313744,-0.225927,0.997174,0.738438,0.200458
HorzBreak,-0.205131,-0.449627,0.315139,-0.148164,0.240898,1.000000,0.800259,-0.995290,0.242679,0.229091,0.223448
SpinAxis,-0.242976,-0.040127,0.293251,-0.218405,0.313744,0.800259,1.000000,-0.769700,0.311570,0.277810,0.449991
pfxx,0.196533,0.514619,-0.308828,0.132162,-0.225927,-0.995290,-0.769700,1.000000,-0.228436,-0.215774,-0.130960
pfxz,-0.702326,0.026283,0.716080,-0.144478,0.997174,0.242679,0.311570,-0.228436,1.000000,0.697113,0.195385
VertApprAngle,-0.662862,-0.002670,0.672893,-0.050363,0.738438,0.229091,0.277810,-0.215774,0.697113,1.000000,0.177499


In [ ]:
df6 = df4.corr()
csv = df6.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/HitTypeCorr*.csv", index=False)

In [ ]:
import statistics

# 'target_column' is the name of the column with 0s and 1s, and 'predictor_columns' is a list of predictor column names.

def compute_statistics_by_target(df, target_column, predictor_columns):
    # Group the dataframe by the target variable and calculate statistics for each group
    grouped = df.groupby(target_column)

    # Initialize a dictionary to store the statistics for each group
    statistics_dict = {}

    for target_value, group in grouped:
        # Calculate the means for each predictor column
        means = group[predictor_columns].mean()

        # Calculate the standard deviations for each predictor column
        std_devs = group[predictor_columns].std()

        # Calculate the medians for each predictor column
        medians = group[predictor_columns].median()

        # Calculate the 25th percentile for each predictor column
        percentile_25 = group[predictor_columns].quantile(0.25)

        # Calculate the 75th percentile for each predictor column
        percentile_75 = group[predictor_columns].quantile(0.75)

        # Combine the means and standard deviations for each group into a single row
        row = means.tolist() + std_devs.tolist() + medians.tolist() + percentile_25.tolist() + percentile_75.tolist()

        # Add the row to the statistics dictionary
        statistics_dict[target_value] = row

    # Convert the dictionary to a dataframe and set the index
    statistics_df = pd.DataFrame.from_dict(statistics_dict, orient='index', columns=predictor_columns +
                                        [f'Std_Dev_{col}' for col in predictor_columns] +
                                        [f'Median_{col}' for col in predictor_columns] +
                                        [f'Percentile_25_{col}' for col in predictor_columns] +
                                        [f'Percentile_75_{col}' for col in predictor_columns])


    return statistics_df

# Example usage:
target_column_name = 'isHit'
predictor_columns_list = ['pitchType', 'pitcherHand', 'hitterHand', 'ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'HitSpinRate', 'Distance', 'Bearing', 'HangTime', 'MaxHeight', 'ContactPositionX', 'ContactPositionY', 'ContactPositionZ']
statistics_table = compute_statistics_by_target(df, target_column_name, predictor_columns_list)

print(statistics_table)

In [ ]:
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/HitTypeANOVA*.csv", index=False)

# Processes data for model (updated)


In [ ]:
df5 = df3[['pitchType', 'pitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'VertApprAngle', 'HorzApprAngle']]

In [ ]:
print(len(df5[df5['pitchType'] == 1]))
print(len(df5[df5['pitchType'] == 2]))
print(len(df5[df5['pitchType'] == 3]))
print(len(df5[df5['pitchType'] == 4]))

1062310
454762
223975
138948


In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame df5 with a target column 'pitchType'

# Create separate DataFrames for each class
class_1 = df5[df5['pitchType'] == 1]
class_2 = df5[df5['pitchType'] == 2]
class_3 = df5[df5['pitchType'] == 3]
class_4 = df5[df5['pitchType'] == 4]

# Calculate the desired undersampling factors for the majority class (Fastball)
# and keep the minority classes as they are (ratio = 1)
desired_ratios = {
    1: 1/3,    # For Fastball, keep the minority class size unchanged
    2: 3/5,      # For Slider, keep the minority class size unchanged
    3: 1,      # For ChangeUp, keep the minority class size unchanged
    4: 1.5       # For Curveball, keep the minority class size unchanged
}

# Undersample the majority class to match the desired ratio and calculate example weights
undersampled_classes = []

for class_label in [1, 2, 3, 4]:
    class_df = df5[df5['pitchType'] == class_label]
    desired_ratio = desired_ratios[class_label]

    if class_label == 4:
      undersampled_class = resample(class_df,
                                  replace=True,
                                  n_samples=int(len(class_df) * desired_ratio),
                                  random_state=42)
      print(str(class_label) + " " + str(len(class_df) * desired_ratio))

    else:
        # Undersample the majority classes (Non-curveball pitches)
        undersampled_class = resample(class_df,
                                      replace=False,
                                      n_samples=int(len(class_df) * desired_ratio),
                                      random_state=42)
        print(str(class_label) + " " + str(len(class_df) * desired_ratio))

    # Calculate example weights based on the undersampling factor
    undersampled_class['example_weight'] = 1 / desired_ratio

    undersampled_classes.append(undersampled_class)

# Concatenate the undersampled classes
balanced_df = pd.concat(undersampled_classes)

# Shuffle the data to avoid order effects
balanced_df = balanced_df.sample(frac=1, random_state=42)

# Split the dataset into features and target
rel_data = balanced_df[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'VertApprAngle', 'HorzApprAngle', 'pitcherHand']].to_numpy()
classification = balanced_df['pitchType'].to_numpy()

# Determine the train-test split
sampledTotal = len(balanced_df)
trainTotal = int(0.8 * sampledTotal)
print("Train Total: " + str(trainTotal))

# Split data into training and testing sets
x_train = rel_data[:trainTotal]
x_test = rel_data[trainTotal:]
y_train = classification[:trainTotal]
y_test = classification[trainTotal:]
sample_weights_train = balanced_df['example_weight'][:trainTotal]
sample_weights_test = balanced_df['example_weight'][trainTotal:]


1 354103.3333333333
2 272857.2
3 223975
4 208422.0
Train Total: 847485


In [ ]:
len(balanced_df)

1059357

# Trains for the models (Old)

In [ ]:
# adjust this based on pitch type: Fastball = 1, Slider = 2, ChangeUp = 3, Curveball = 4

minority_class = df5[df5['pitchType'] == 1]
majority_class = df5[df5['pitchType'] != 1]

print("majority class is: " + str(len(majority_class)))
print("minority class is: " + str(len(minority_class)))

print("total is: " + str(len(majority_class)+len(minority_class)))

print("Majority to Minority Ratio: " + str((len(majority_class))/len(minority_class)))

majority class is: 817685
minority class is: 1062310
total is: 1879995
Majority to Minority Ratio: 0.7697235270307161


In [ ]:
# Determine the downsampling factor
downsampling_factor = 1

# resamples the class to have a even distribution of hits and non hits
from sklearn.utils import resample
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      #n_samples=len(minority_class) * downsampling_factor,  # match minority class size
                                      random_state=42)  # reproducible results
# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class, majority_class_downsampled])

# Calculate the example weights
example_weight = 1 / downsampling_factor
# Assign the weights to the downsampled majority class
balanced_df.loc[balanced_df['pitchType'] != 1, 'example_weight'] = example_weight
balanced_df['example_weight'].fillna(1, inplace=True)  # Fill NaN with 1 for minority class

In [ ]:
# Shuffle the data to avoid order effects
# figure why this don't work
balanced_df = balanced_df.sample(frac=1, random_state=42)


In [ ]:
rel_data = balanced_df[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'pfxx', 'pfxz', 'VertApprAngle', 'HorzApprAngle', 'pitcherHand']].to_numpy()
classification = balanced_df['pitchType'].to_numpy()

In [ ]:
minority = balanced_df[balanced_df['pitchType'] == 1]
majority = balanced_df[balanced_df['pitchType'] != 1]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

sampledTotal = len(majority) + len(minority)
trainTotal = int(0.75*sampledTotal)

majority class is: 817685
minority class is: 1062310
total is: 1879995


In [ ]:
# Split data into features and target
X = balanced_df.drop(['pitchType', 'example_weight'], axis=1)
y = balanced_df['pitchType']
# x = np.unique(classification, return_inverse=True)
# y = np.unique(classification, return_inverse=True)
sample_weights = balanced_df['example_weight']

# Split data into training and testing sets
x_train = X[:trainTotal]
x_test = X[trainTotal:]
y_train = y[:trainTotal]
y_test = y[trainTotal:]

from sklearn import preprocessing

# Standardize the features
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Makes the Model



In [ ]:
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = [
    "Decision Tree",
    "Random Forest"
]

classifiers = [
    #DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=None, n_estimators=50)
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['fastball', 'slider', 'changeup', 'curveball']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    fastball       0.98      0.97      0.97     70968
      slider       0.95      0.94      0.95     54292
    changeup       0.95      0.97      0.96     44917
   curveball       0.96      0.97      0.96     41695

    accuracy                           0.96    211872
   macro avg       0.96      0.96      0.96    211872
weighted avg       0.96      0.96      0.96    211872



In [ ]:
print(accuracies)

[0.962010081558677]


In [ ]:
print(accuracies)

[0.9571623863274095]


# Exports the Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/SabermetricModels')

# export the trained model to disk
joblib.dump(clf, 'PitchModel.joblib')

['PitchModel.joblib']